# Behaviour based content clustering

## Kernel - 3 for DonorsChoose.org

In this kernel, we will examine the behavior of people and do the content based filtering in a very intelligible way that it will perform at its best. The main idea behind this kernel will be to examine how they behave is changing from one donation to the next donation and make categories in such a way that it accounts for the gradual change in the behavior of people and give best possible results for a content-based filter.

The idea behind this kernel is that if the behavior is changing drastically then one can't recommend similar content to last donation. If we are going to suggest similar content we should ensure that the behaviour is not changing ( or not changing across the variables that we are selecting for content suggestion) or we have to keep checking if the behaviour is different every once in a while and should make changes in the variables such that the content suggested by us will be the optimum quality of content a donor is interested in.

### This analysis will have two parts - 
                    - Input prep such that behaviour is same for selected variables
                    - Clustering based recommendation engine - for suggesting donors for projects
                    




## Workflow diagram

![](https://lh3.googleusercontent.com/-ZvnzeqXmNak/WyoYgBgesTI/AAAAAAAAvcU/cr4r0TFfMEMnbXGRA1q_vBdSK3keIsJZQCL0BGAYYCw/h1183/2018-06-20.png)

In [1]:
# importing packages 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')
import time
import datetime
from scipy.sparse.linalg import svds
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_notebook, show
output_notebook()
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
import plotly.tools as tls
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.cluster import MiniBatchKMeans
from functools import reduce
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

## Loading Data
We will load the data files, and merge the projects file with donations file so that we can have the information that what type of projects gets the attention of what type of donors. We will filter the donors which has donoated twice and divide the datset into two dataframes - dataframe having first donation and dataframe having second donations. Between these two dataframe we will analyse the change in behaviour between first and second donation and will make our hypothesis based on the results.

In [2]:
donations = pd.read_csv("../input/Donations.csv", encoding='latin-1')
donors = pd.read_csv("../input/Donors.csv", encoding='latin-1')
projects = pd.read_csv("../input/Projects.csv", encoding='latin-1')
resources = pd.read_csv("../input/Resources.csv", encoding='latin-1')
schools = pd.read_csv("../input/Schools.csv", encoding='latin-1')
teachers = pd.read_csv("../input/Teachers.csv", encoding='latin-1')

## Input Prep

In [3]:
# Find the donations count to get the number of donations
donations_count = pd.DataFrame(
    donations.groupby("Donor ID")['Project ID'].count()
)
donations_count.reset_index(inplace = True)
donations_count.rename(columns = {'Project ID':'donation_count'},
                       inplace = True)
donations_count.head()

In [4]:
donors_having_two_donations = \
    donations_count.loc[donations_count['donation_count'] == 2]
# Donors having two donations has data of donors who has donated twice

data_ = donors_having_two_donations.merge(donations, on='Donor ID',
        how='right')
# data_ dataframe has donations data for donors having two donations

data_.sort_values(['Donor ID', 'Donation Received Date'], inplace=True)
# Sort these values so that these two donoations are adjecent to each other


odd = [x for x in list(range(0, 274902)) if x%2 ==1] 
even = [x for x in list(range(0, 274902)) if x%2 ==0] 
first = data_.loc[even]  # data having first donations of data
second = data_.loc[odd]  # data for second donations data

In [7]:
# merging projects and schools file in first and second donations
first_df = projects.merge(first, left_on='Project ID',
                          right_on='Project ID', how='right')
second_df = projects.merge(second, left_on='Project ID',
                           right_on='Project ID', how='right')

first_df = first_df.merge(schools, left_on='School ID',
                          right_on='School ID', how='left')
second_df = second_df.merge(schools, left_on='School ID',
                            right_on='School ID', how='left')

# Save the copy of first_df and second_df
first_df_copy = first_df.copy()
second_df_copy = second_df.copy()

## Analysing and Adjusting Behaviour 
For analyzing behavior, we will have to check for each variable that if a donor who had a preference A in past donation, what is his/her preference for current donation. It can be made clear with two examples
                - A) Donors that were donated to the rural school in past, are donating X% to rural and rest y% to urban
                - B) Donor donating to state A is donating to state A only.
In these two examples, in example A, the behavior is changing but in example B the behavior is constant. we have to analyze for all the people and see if the significant number of people's behavior is changing then we will merge the two levels of that parameters (adjust that parameters) and that will/may result in making the behavior constant regarding that parameter. If e continue to do so for all the variables we use, we will reach to a point, where across the adjusted parameters, the behavior change is least.

** For examining the behavior change, I will be using simple confusion matrices (heatmaps of past vs current behavior). One can use any other kind of plots like Sankey or alluvial plots. This will also show that a simple plot like confusion matrix can be a basis for making proper input**

In example shown below,  following variables will be adjusted - 
                    - A) Project Grade Level Category
                    - B) Project Resource Category
                    - C) School State
                    - D) School Metro Type
                    - E) School Percentage Free Lunch - will be making categories for this
                    - F) Project Cost - will be making categories for this
                    

In [8]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
        Input - 
                cm - confusion matrics
                classes - classes
        Output - 
                Plots confusion matrix
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
    #    print('Confusion matrix, without normalization')

    #print(cm)
    fig = plt.figure(figsize = (14,14))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('Current behaviour')
    plt.xlabel('Past behaviour')
    plt.show()

In [9]:
def check_behaviour_shift(var, first_df, second_df):
    """
    Function to check the behaviour shift in the first
    donation to second donation
        Input - 
                var - variables to be plotted 
                first_df - past donations dataframe
                second_df - current donations dataframe
    """
    first = [str(x) for x in first_df[var].values]
    second = [str(x) for x in second_df[var].values]
    label = list(set(first))
    print(var)
    cnf_matrix = confusion_matrix(first, second, label)
    cnf_matrix

    plot_confusion_matrix(cnf_matrix, label,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Accent)
    return

In [10]:
def variable_adjustment(
    first_df,
    second_df,
    var,
    list_values,
    ):
    """Function to merge the distinct values of variable and make them as one"""

    first = first_df.copy()
    second = second_df.copy()
    new_value = ''
    for values in list_values:
        new_value = new_value + ' ' + values

    # first.loc[(first[var].isin(list_values)), var] = new_value

    first[var] = np.where(first[var].isin(list_values), new_value,
                          first[var])

    # second.loc[(second[var].isin(list_values)), var] = new_value

    second[var] = np.where(second[var].isin(list_values), new_value,
                           second[var])
    return (first, second)

#### 1. Project Grade Level Category

In [11]:
check_behaviour_shift('Project Grade Level Category', first_df, second_df)

In [12]:
# first Adjustment
list_values = ['Grades PreK-2', 'Grades 6-8']
(first_df_1, second_df_1) = variable_adjustment(first_df, second_df,
        'Project Grade Level Category', list_values)
check_behaviour_shift('Project Grade Level Category', first_df_1,
                      second_df_1)

# second Adjustment
list_values2 = ['Grades 3-5', 'Grades 9-12']
(first_df_2, second_df_2) = variable_adjustment(first_df_1,
        second_df_1, 'Project Grade Level Category', list_values2)
check_behaviour_shift('Project Grade Level Category', first_df_2,
                      second_df_2)

** Its is clear from the above plots that this variable should not be used for clustering as merging all variables only will result in consistant behaviour**

#### 2 Project Resource Category

In [13]:
check_behaviour_shift('Project Resource Category', first_df_copy, second_df_copy)

In [14]:
# behaviour adjustment 
list_values3 = ['Others', 'Supplies', 'Books', 'Technology']
var = 'Project Resource Category'
(first_df_3, second_df_3) = variable_adjustment(first_df, second_df,
        var, list_values3)
check_behaviour_shift(var, first_df_3, second_df_3)

After adjustment we can see that the category - "Others Supplies Books Technology" has most filled and is on diagonal so behaviour is change the least from past to current

#### 3. School State

In [15]:
check_behaviour_shift('School State', first_df_copy, second_df_copy)

In [16]:
# behaviour adustment
list_values4 = [
    'Illinois',
    'Florida',
    'Hawaii',
    'New York',
    'California',
    'Washington',
    'North Carolina',
    'Texas',
    'Georgia',
    ]
var = 'School State'
(first_df_4, second_df_4) = variable_adjustment(first_df_3,
        second_df_3, var, list_values4)
check_behaviour_shift(var, first_df_4, second_df_4)

After adjustment the most occupied entry is on diagonal so behave is consistant

#### 4. School Metro Type

In [17]:
check_behaviour_shift('School Metro Type', first_df_copy, second_df_copy)

In [18]:
list_values5 = ['urban', 'unknown', 'suburban']
var = 'School Metro Type'
(first_df_5, second_df_5) = variable_adjustment(first_df_4,
        second_df_4, var, list_values5)
check_behaviour_shift(var, first_df_5, second_df_5)

In [19]:
# Variables to be digitize - 1. School free lunch and 2. project cost
fig, ax = plt.subplots(figsize = (18,9),ncols=2, sharex=False, sharey=False)
sns.despine()
sns.distplot(first_df['School Percentage Free Lunch'].dropna(), ax=ax[0])
sns.distplot(np.log(first_df['Project Cost'].dropna()+1), ax=ax[1])
plt.show()

#### Digitizing % free lunch and project cost
We will be using the log of project cost

In [20]:
prec_free_lunch_bins = [
    0,
    20,
    40,
    60,
    80,
    100,
    ]
first_df_5['perc_freee_lunch'] = \
    np.digitize(first_df_5['School Percentage Free Lunch'].values,
                prec_free_lunch_bins)
second_df_5['perc_freee_lunch'] = \
    np.digitize(second_df_5['School Percentage Free Lunch'].values,
                prec_free_lunch_bins)

proj_cost_bins = [
    0,
    4,
    6,
    8,
    10,
    20,
    ]
first_df_5.loc[:, 'log_proj_cost'] = np.log(first_df_5['Project Cost'
        ].dropna() + 1)
second_df_5.loc[:, 'log_proj_cost'] = np.log(second_df_5['Project Cost'
        ].dropna() + 1)
first_df_5['proj_cost'] = np.digitize(first_df_5['log_proj_cost'
        ].values, proj_cost_bins)
second_df_5['proj_cost'] = np.digitize(second_df_5['log_proj_cost'
        ].values, proj_cost_bins)

first_df_5.head()

#### 5. proj_cost

In [21]:
check_behaviour_shift('proj_cost', first_df_5, second_df_5)

In [22]:
list_values6 = ['2', '3']
var = 'proj_cost'
(first_df_6, second_df_6) = variable_adjustment(first_df_5,
        second_df_5, var, list_values6)
check_behaviour_shift(var, first_df_6, second_df_6)

#### 6. perc_freee_lunch

In [23]:
check_behaviour_shift("perc_freee_lunch", first_df_6, second_df_6)

In [24]:
list_values7 = ['4', '5', '2', '3']
var = 'perc_freee_lunch'
(first_df_7, second_df_7) = variable_adjustment(first_df_6,
        second_df_6, var, list_values7)
check_behaviour_shift(var, first_df_7, second_df_7)

# Content based clustering on adjusted project attributes

 ### Create dummy for clustering 

In [25]:
list_var = [
    'Project Grade Level Category',
    'Project Resource Category',
    'School State',
    'School Metro Type',
    'proj_cost',
    'perc_freee_lunch',
    ]

full_data = pd.concat([first_df_7, second_df_7], axis=0)
full_data.shape
full_data.reset_index(inplace = True)

list_7 = first_df_7.columns

full_data_2 = pd.get_dummies(data=full_data, columns=list_var)
list_8 = full_data_2.columns
dummies_col = [x for x in list_8 if x not in list_7]
first_df_8 = full_data_2.head(274902 // 2)
first_df_8[dummies_col].head()

In [27]:
def Assign_cluster(
    data1,
    data2,
    list_var,
    clus_size,
    ):
    """Function to assign cluster using k means clustering
        Input - 
                data1/2 - dataframes having adjusted variables
                list_var - list of variables for clustering
                clus_size - Number of clusters
        Output - 
                data - this data has cluster number 
                
    """

    t0 = time.time()
    print(data1.shape[0])
    data = pd.concat([data1, data2], axis=0)
    print(data.shape[0])
    kmeans = MiniBatchKMeans(n_clusters=clus_size,
                             batch_size=10000).fit(data[list_var])
    data1.loc[:, 'cluster'] = kmeans.predict(data1[list_var])
    data2.loc[:, 'cluster'] = kmeans.predict(data2[list_var])
    t1 = time.time()
    print('Time taken in clustering is {}'.format(t1 - t0))
    return (data1, data2)


(data_1, data_2) = Assign_cluster(first_df_8, full_data_2.loc[137452:],
                                  dummies_col, 120)
data_1.head()
   

#### Vasualize number of donors per cluster

In [28]:
# Checking the distribution across clusters
plt.hist(data_2['cluster'].values)
plt.show()

In [29]:
def Prediction_on_cluster_based_engine(train_df, test_df, n):
    """Function to predict using the cosine distnace based engine"""

    test_df['reco_ids'] = ''

    def predictions(row, train_df=train_df):
        ids = list(train_df.loc[train_df['cluster'] == row['cluster'
                   ]]['Donor ID'])[0:n]
        return ids

    test_df['reco_ids'] = test_df.apply(lambda row: predictions(row),
            axis=1)
    return test_df


test_clus = Prediction_on_cluster_based_engine(data_1,
        data_2[5000:5500], 5000)
test_clus.head()

In [30]:
def cosine_sim_reco(
    data,
    new_project_profile,
    var_list,
    dummies_col,
    n,
    ):
    """Function to calculate cosine distance between two donors"""

    t0 = time.time()
    data_copy = data.copy()
    data_copy.reset_index(inplace=True)

    def get_project_profile(row, proj=new_project_profile,
                            var_list=var_list):
        profile = np.array(row[dummies_col])
        proj = np.array(proj)
        cs = cosine_similarity(profile.reshape(1, -1), proj.reshape(1,
                               -1))
        return cs[0][0]

    data_copy['cosine_'] = data_copy.apply(lambda row: \
            get_project_profile(row), axis=1)
    data_copy = data_copy.sort_values('cosine_', ascending=False)
    recommended_donors = data_copy['Donor ID'].loc[0:n]
    t1 = time.time()
    print('Time taken in clustering is {}'.format(t1 - t0))
    return recommended_donors
  

In [31]:
def Prediction_on_content_based_filtering(
    train_df,
    test_df,
    dummies_col,
    list_var,
    n,
    ):
    """Function to predict using the cosine distnace based engine"""

    test = test_df.copy()
    test.reset_index(inplace=True)
    test['recommended_IDs'] = ''
    print("Shape of test is {}".format(test.shape[0]))
    for i in list(range(test.shape[0])):
        pro_prof = test[dummies_col].loc[i].values
        recommended_donors = cosine_sim(train_df, pro_prof, list_var,
                dummies_col, n)
        pred_ = list(recommended_donors)
        test['recommended_IDs'][i] = pred_
    return test

In [33]:
def reco_success_rate(row):
    """Function to calculate how many reco must have succeeded if we used this model
        input - df - test_ensemble dataframe
                k - number of reco to conside
        Ouutput - % successful reco email fire
    """
    success = 0
    #print(row['recommended_IDs'])
    if row['Donor ID'] in row['reco_ids']:
        success =1 
    return(success)

def Mean_success_rate(df):
    """Function to calculate the MSR"""
    df['success'] = df.apply(lambda row:reco_success_rate(row), axis =1)
    success = 1.0*df['success'].sum()
    total = df.shape[0]
    return(np.float((success/total)))

In [34]:
Mean_success_rate(test_clus)

## Hyperparameter tuning - Number of clusters 

In [39]:
list_clus = [20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 130]
number_of_pred = [2000, 3000, 4000, 5000, 6000]
run = False # make it True and when you run the notebook in your local machine
if run:
    t0 = time.time()
    for clus_size in list_clus:
        succ_vs_clus_size ={}
        success = []
        for num_pred in number_of_pred:
            (data_1, data_2) = Assign_cluster(first_df_8, full_data_2.loc[137452:],
                                          dummies_col, clus_size)
            test_clus = Prediction_on_cluster_based_engine(data_1,
                data_2.sample(10000), num_pred)
            succ_ = Mean_success_rate(test_clus)
            print("Success :", succ_)
            print("Number of :", num_pred)
            print("="*52)
            success.append(succ_)
        print(success)
        succ_vs_clus_size[clus_size] = success
    t1 =time.time()
    print("Time taken in hyperparamter optimization is {}".format(t1-t0))

    succ_vs_clus_size    

## Plot of Success rate (y axis) vs number of prediction i.e. k (x axis)

In [41]:
# Results hgas been copied for plotting
_x = np.array(number_of_pred)
_y0 = np.array([0.0974, 0.1541, 0.1978, 0.2576, 0.3073])
_y1 = np.array([0.1401, 0.2032, 0.2651, 0.3064, 0.3052])
_y2 = np.array([0.1713, 0.2441, 0.2841, 0.2774, 0.2871])
_y3 = np.array([0.2031, 0.2588, 0.2634, 0.2662, 0.2617])
_y4 = np.array([0.2179, 0.2517, 0.2565, 0.2545, 0.2527])
_y5 = np.array([0.2378, 0.2435, 0.246, 0.2341, 0.2427])
_y6 = np.array([0.2292, 0.2368, 0.2395, 0.2323, 0.2388])
_y7 = np.array([0.2227, 0.2278, 0.2264, 0.2223, 0.2256])
_y8 = np.array([0.2199, 0.2165, 0.2196, 0.2167, 0.2219])
_y9 = np.array([0.2053, 0.203, 0.2095, 0.2089, 0.2074])
_y10 = np.array([0.2087, 0.2008, 0.2078, 0.2028, 0.2087])

# Create traces
trace0 = go.Scatter(
    x = _x,
    y = _y0,
    mode = 'lines+markers',
    name = 'cluster size 20'
)
trace1 = go.Scatter(
    x = _x,
    y = _y1,
    mode = 'lines+markers',
    name = 'cluster size 30'
)
trace2 = go.Scatter(
    x = _x,
    y = _y2,
    mode = 'lines+markers',
    name = 'cluster size 40'
)
trace3 = go.Scatter(
    x = _x,
    y = _y3,
    mode = 'lines+markers',
    name = 'cluster size 50'
)
trace4 = go.Scatter(
    x = _x,
    y = _y4,
    mode = 'lines+markers',
    name = 'cluster size 60'
)
trace5 = go.Scatter(
    x = _x,
    y = _y5,
    mode = 'lines+markers',
    name = 'cluster size 70'
)
trace6 = go.Scatter(
    x = _x,
    y = _y6,
    mode = 'lines+markers',
    name = 'cluster size 80'
)
trace7 = go.Scatter(
    x = _x,
    y = _y7,
    mode = 'lines+markers',
    name = 'cluster size 90'
)

trace8 = go.Scatter(
    x = _x,
    y = _y8,
    mode = 'lines+markers',
    name = 'cluster size 100'
)
trace9 = go.Scatter(
    x = _x,
    y = _y9,
    mode = 'lines+markers',
    name = 'cluster size 120'
)
trace10 = go.Scatter(
    x = _x,
    y = _y10,
    mode = 'lines+markers',
    name = 'cluster size 130'
)

data = [trace0, trace1, trace2, trace3,trace4, trace5, trace6, trace7, trace8, trace9, trace10]

fig = go.Figure(data=data)
iplot(fig, filename='line-mode')

## Final recommendation (which engine to use ? - it's a subjective decision)
- **ROI** - It depends on the vendor DonorsChoose selects for Email marketing. DonorsChoose must maximize ROI such that money they spend on email marketing should maximize their donations. e.g. if Donors Choose vendor X for email marketing to charge them for \$0.01 per mail then they should use Engine 1 as engine 1 gives us >50/% success rate for every 1000 emails i.e. \$10 per project will give us 50% chance of success while if DonorsChoose's vendor X charges a fixed amount below 5000 emails then they should use engine 2 or 3 as they give us 30% success rate while keeping the cope of mailing a little bit different donors thus diversify the campaign.

- **Computation Cost** - Engine 1 uses the full matrix at one go and since we know that the size of SVD is 2.2M X 100K and the order of SVD is O(min{mn2,m2n}) which will lead to high computation costs (though it will give best results if trained on full data). The computation costs can go up to $0.5 per hour so that case they may use engine 2 or 3 and little low accuracies but save run cost. Finally a subjective decision.




- **Re-training model on real-time** - It is possible that DonorsChoose has to retrain recommender engine every few days as new data is coming and the engine will perform best when it is updated. In that case, it makes more sense to use engine 2 or 3 as they can be trained within seconds.

PS - DonorsChoose can always use results from all the three engines and send emails based on them.
